# Rf 5 0 8_Listsetmanip
Organization and simultaneous fits: RooArgSet and RooArgList tools and tricks



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:56 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "RooPlot.h"
#include "RooArgSet.h"
#include "RooArgList.h"
#include "RooCategory.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Create dummy objects
 ---------------------------------------

Create some variables

In [3]:
RooRealVar a("a", "a", 1, -10, 10);
RooRealVar b("b", "b", 2, -10, 10);
RooRealVar c("c", "c", 3, -10, 10);
RooRealVar d("d", "d", 4, -10, 10);
RooRealVar x("x", "x", 0, -10, 10);
c.setError(0.5);
a.setConstant();
b.setConstant();


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Create a category

In [4]:
RooCategory e("e", "e");
e.defineType("sig");
e.defineType("bkg");

Create a pdf

In [5]:
RooGaussian g("g", "g", x, a, b);

Creating, filling rooargsets
 -------------------------------------------------------

A rooargset is a set of rooabsarg objects. each object in the set must have
 a unique name

Set constructors exists with up to 9 initial arguments

In [6]:
RooArgSet s(a, b);

At any time objects can be added with add()

In [7]:
s.add(e);

Add up to 9 additional arguments in one call

In [8]:
s.add(RooArgSet(c, d));

Sets can contain any type of rooabsarg, also pdf and functions

In [9]:
s.add(g);

Remove element d

In [10]:
s.remove(d);

Accessing rooargset contents
 -------------------------------------------------------

You can look up objects by name

In [11]:
RooAbsArg *aptr = s.find("a");

Construct a subset by name

In [12]:
RooArgSet *subset1 = (RooArgSet *)s.selectByName("a,b,c");

Construct asubset by attribute

In [13]:
RooArgSet *subset2 = (RooArgSet *)s.selectByAttrib("Constant", kTRUE);

Construct the subset of overlapping contents with another set

In [14]:
RooArgSet s1(a, b, c);
RooArgSet s2(c, d, e);
RooArgSet *subset3 = (RooArgSet *)s1.selectCommon(s2);

Owning rooargsets
 ---------------------------------

Create a rooargset that owns its components
 A set either owns all of its components or none,
 so once addOwned() is used, add() can no longer be
 used and will result in an error message

In [15]:
RooRealVar *ac = (RooRealVar *)a.clone("a");
RooRealVar *bc = (RooRealVar *)b.clone("b");
RooRealVar *cc = (RooRealVar *)c.clone("c");

RooArgSet s3;
s3.addOwned(RooArgSet(*ac, *bc, *cc));

Another possibility is to add an owned clone
 of an object instead of the original

In [16]:
s3.addClone(RooArgSet(d, e, g));

A clone of a owning set is non-owning and its
 contents is owned by the originating owning set

In [17]:
RooArgSet *sclone = (RooArgSet *)s3.Clone("sclone");

To make a clone of a set and its contents use
 the snapshot method

In [18]:
RooArgSet *sclone2 = (RooArgSet *)s3.snapshot();

If a set contains function objects, only the head node
 is cloned in a snapshot. To make a snapshot of all
 servers of a function object do as follows. The result
 of a RooArgSet snapshot with deepCloning option is a set
 of cloned objects, and all their clone (recursive) server
 dependencies, that together form a self-consistent
 set that is free of external dependencies

In [19]:
RooArgSet *sclone3 = (RooArgSet *)s3.snapshot(kTRUE);

Set printing
 ------------------------

Inline printing only show list of names of contained objects

In [20]:
cout << "sclone = " << (*sclone) << endl;

sclone = (a,b,c,d,e,g)


Plain print shows the same, prefixed by name of the set

In [21]:
sclone->Print();

RooArgSet::sclone = (a,b,c,d,e,g)


Standard printing shows one line for each item with the items name, class name and value

In [22]:
sclone->Print("s");

  1) RooRealVar:: a = 1
  2) RooRealVar:: b = 2
  3) RooRealVar:: c = 3 +/- 0.5
  4) RooRealVar:: d = 4
  5) RooCategory:: e = sig(idx = 0)

  6) RooGaussian:: g = 0.882497


Verbose printing adds each items arguments, address and 'extras' as defined by the object

In [23]:
sclone->Print("v");

  1) 0x7f2ee459ecb0 RooRealVar:: a = 1 C  L(-10 - 10)  "a"
  2) 0x7f2ee4470d50 RooRealVar:: b = 2 C  L(-10 - 10)  "b"
  3) 0x7f2ee446fd60 RooRealVar:: c = 3 +/- 0.5  L(-10 - 10)  "c"
  4) 0x7f2ee45cdd30 RooRealVar:: d = 4  L(-10 - 10)  "d"
  5) 0x7f2ee45ae060 RooCategory:: e = sig(idx = 0)
  "e"
  6) 0x7f2ee44dca90 RooGaussian:: g[ x=x mean=a sigma=b ] = 0.882497  "g"


Using rooarglists
 ---------------------------------

List constructors exists with up to 9 initial arguments

In [24]:
RooArgList l(a, b, c, d);

Lists have an explicit order and allow multiple arguments with the same name

In [25]:
l.add(RooArgList(a, b, c, d));

Access by index is provided

In [26]:
RooAbsArg *arg4 = l.at(4);